In [1]:
import csv
from datetime import timedelta
import gurobipy as gp 
from gurobipy import GRB
import networkx as nx
import random
import matplotlib.pyplot as plt
import inspect
from collections import defaultdict

from models import simple_mpc, mpc_duration_constr, lazy, column_generation
from helper import Service, hhmm2mins, mins2hhmm, fetch_data, draw_graph_with_edges, node_legal, no_overlap, create_duty_graph, extract_nodes, generate_paths, roster_statistics, solution_verify, solving_RMLP_art_vars_final

In [2]:
services, service_dict = fetch_data('./StepBackServices.csv')
graph = create_duty_graph(services)

In [29]:
init_duties, duty_count = mpc_duration_constr(graph, service_dict, time_limit = 30, show_logs = True, show_duties = False, show_roster_stats = True)

Set parameter TimeLimit to value 30
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 5 4600H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
TimeLimit  30

Optimize a model with 104364 rows, 68020 columns and 335422 nonzeros
Model fingerprint: 0x4522e447
Variable types: 34478 continuous, 33542 integer (33542 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Presolve removed 4061 rows and 1230 columns
Presolve time: 1.24s
Presolved: 100303 rows, 66790 columns, 373298 nonzeros
Variable types: 33424 continuous, 33366 integer (33366 binary)
Found heuristic solution: objective 900.0000000
Deterministic concurrent LP optimizer: primal and dual simplex
Showing primal log only...

Concurrent spin time: 0.00s

Solved 

In [30]:
duty_count

146

In [41]:
import copy
init_duties_copy = copy.deepcopy(init_duties)

In [33]:
selected_duties, dual_values, selected_duties_vars = solving_RMLP_art_vars_final(services, init_duties)
# selected_duties

Optimal solution found
Positive Duties, 0:  146
x0 1.0
x1 1.0
x2 1.0
x3 1.0
x4 1.0
x5 1.0
x6 1.0
x7 1.0
x8 1.0
x9 1.0
x10 1.0
x11 1.0
x12 1.0
x13 1.0
x14 1.0
x15 1.0
x16 1.0
x17 1.0
x18 1.0
x19 1.0
x20 1.0
x21 1.0
x22 1.0
x23 1.0
x24 1.0
x25 1.0
x26 1.0
x27 1.0
x28 1.0
x29 1.0
x30 1.0
x31 1.0
x32 1.0
x33 1.0
x34 1.0
x35 1.0
x36 1.0
x37 1.0
x38 1.0
x39 1.0
x40 1.0
x41 1.0
x42 1.0
x43 1.0
x44 1.0
x45 1.0
x46 1.0
x47 1.0
x48 1.0
x49 1.0
x50 1.0
x51 1.0
x52 1.0
x53 1.0
x54 1.0
x55 1.0
x56 1.0
x57 1.0
x58 1.0
x59 1.0
x60 1.0
x61 1.0
x62 1.0
x63 1.0
x64 1.0
x65 1.0
x66 1.0
x67 1.0
x68 1.0
x69 1.0
x70 1.0
x71 1.0
x72 1.0
x73 1.0
x74 1.0
x75 1.0
x76 1.0
x77 1.0
x78 1.0
x79 1.0
x80 1.0
x81 1.0
x82 1.0
x83 1.0
x84 1.0
x85 1.0
x86 1.0
x87 1.0
x88 1.0
x89 1.0
x90 1.0
x91 1.0
x92 1.0
x93 1.0
x94 1.0
x95 1.0
x96 1.0
x97 1.0
x98 1.0
x99 1.0
x100 1.0
x101 1.0
x102 1.0
x103 1.0
x104 1.0
x105 1.0
x106 1.0
x107 1.0
x108 1.0
x109 1.0
x110 1.0
x111 1.0
x112 1.0
x113 1.0
x114 1.0
x115 1.0
x116 1.0
x117 1.0


In [ ]:
def generate_new_column(graph, service_dict, current_duties, dual_values, method = "topological sort"):
    new_duties = []
    reduced_costs = []
    # Update edge weights based on dual values
    for u, v in graph.edges():
        graph[u][v]['weight'] = int(dual_values[u])* service_dict[u] # Adjust edge weight by dual value

In [6]:
from ortools.sat.python import cp_model

load c:\Users\Adel\AppData\Local\Programs\Python\Python313\Lib\site-packages\ortools\.libs\zlib1.dll...
load c:\Users\Adel\AppData\Local\Programs\Python\Python313\Lib\site-packages\ortools\.libs\abseil_dll.dll...
load c:\Users\Adel\AppData\Local\Programs\Python\Python313\Lib\site-packages\ortools\.libs\utf8_validity.dll...
load c:\Users\Adel\AppData\Local\Programs\Python\Python313\Lib\site-packages\ortools\.libs\re2.dll...
load c:\Users\Adel\AppData\Local\Programs\Python\Python313\Lib\site-packages\ortools\.libs\libprotobuf.dll...
load c:\Users\Adel\AppData\Local\Programs\Python\Python313\Lib\site-packages\ortools\.libs\highs.dll...
load c:\Users\Adel\AppData\Local\Programs\Python\Python313\Lib\site-packages\ortools\.libs\ortools.dll...


In [42]:
for u, v in graph.edges():
        if u == -2:
            graph[u][v]['weight'] = 0
        else:
            graph[u][v]['weight'] = int(dual_values[u])* service_dict[u].serv_dur

model = cp_model.CpModel()

incoming_var = defaultdict(list)
outgoing_var = defaultdict(list) 
edge_vars = {} #xij - binary

incoming_adj_list = nx.to_dict_of_lists(graph.reverse())

#Decision Variables
for (i,j) in graph.edges():
    edge_vars[i,j] = model.NewIntVar(0, 1, f'x_{i}_{j}')
    incoming_var[j].append(edge_vars[i,j])
    outgoing_var[i].append(edge_vars[i,j])

#Constraints - none of the existing init_paths should be feasible 
for path in init_duties_copy:
    path.insert(0, -2)
    path.append(-1)
    edges_of_path = []
    for i in range(len(path)-1):
        edges_of_path.append(edge_vars[path[i], path[i+1]])
    model.Add(sum(edges_of_path) <= len(edges_of_path)-1)
    

#Constraints - we should get a path
for node in graph.nodes():
    if node == -2:
        model.Add(sum(outgoing_var[node]) == 1)
    elif node == -1:
        model.Add(sum(incoming_var[node]) == 1)
    else:
        model.Add(sum(incoming_var[node]) == sum(outgoing_var[node]))

#Constraints - dual of path should positive
dual_activated = []
for (i,j) in graph.edges():
    dual_activated.append(edge_vars[i,j]*graph.get_edge_data(i, j)['weight'])
model.Add(sum(dual_activated) > 0)
    

#Constraints - duration of path should be less than 6 hrs
# service_dict[-2] = 0 
duration_activated = []
for (i,j) in graph.edges():
    if i != -2:
        duration_activated.append(edge_vars[i,j]*service_dict[i].serv_dur)
model.Add(sum(duration_activated) <= 6*60)

#Constraints - path should have more than one service  
model.Add(sum(list(edge_vars.values())) >=3)

solver = cp_model.CpSolver()
status = solver.Solve(model)

# 🔹 EXTRACTING FEASIBLE SOLUTION
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    selected_edges = [(i, j) for (i, j) in edge_vars if solver.Value(edge_vars[i, j]) == 1]
    print("Selected Path Edges:", selected_edges)
else:
    print("No feasible solution found.")

Selected Path Edges: [(336, 351), (351, 72), (72, 116), (116, 165), (165, 816), (816, -1), (-2, 336)]


In [ ]:
init_duties
with open("output.txt", "w") as file:
    for sublist in init_duties:
        file.write(" ".join(map(str, sublist)) + "\n")

In [46]:
edge_vars[-2,3]

x_-2_3(0..1)

In [49]:
# generate_paths(outgoing_var, show_paths = True)
for start_path in outgoing_var[-2]:
    current_path = []
    current_path_decision_vars = []
    print(solver.Value(start_path))

1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [ ]:
model = cp_model.CpModel()

for edge in graph.edges():
    model.NewIntVar(0, 1, f'{edge[0]}_{edge[1]}')
    print(edge)

for node in graph.nodes():
    

(336, 6)
(336, 342)
(336, 343)
(336, 9)
(336, 344)
(336, 11)
(336, 13)
(336, 346)
(336, 347)
(336, 15)
(336, 16)
(336, 18)
(336, 349)
(336, 20)
(336, 351)
(336, 22)
(336, 24)
(336, 353)
(336, 26)
(336, 28)
(336, 355)
(336, 30)
(336, 32)
(336, 357)
(336, 34)
(336, 36)
(336, 359)
(336, 38)
(336, 360)
(336, 40)
(336, -1)
(872, 697)
(872, 7)
(872, 698)
(872, 8)
(872, 699)
(872, 10)
(872, 700)
(872, 12)
(872, 701)
(872, 702)
(872, 14)
(872, 703)
(872, 704)
(872, 17)
(872, 705)
(872, 19)
(872, 706)
(872, 707)
(872, 21)
(872, 708)
(872, 23)
(872, 709)
(872, 25)
(872, 710)
(872, 27)
(872, 711)
(872, 29)
(872, 712)
(872, 31)
(872, 713)
(872, 33)
(872, 714)
(872, 715)
(872, 35)
(872, 37)
(872, 716)
(872, 717)
(872, -1)
(873, 347)
(873, 15)
(873, 16)
(873, 18)
(873, 349)
(873, 20)
(873, 351)
(873, 22)
(873, 24)
(873, 353)
(873, 26)
(873, 28)
(873, 355)
(873, 30)
(873, 32)
(873, 357)
(873, 34)
(873, 36)
(873, 359)
(873, 38)
(873, 360)
(873, 40)
(873, 362)
(873, 42)
(873, 44)
(873, 364)
(873, 46)
(